# deCIFer: Metrics & Validity Retrieval  

- Retrieves structural metrics and validity checks from deCIFer-generated CIF data  
- Extracts cell parameters, volumes, and refinement metrics (Rwp, WD)  
- Computes mean, standard deviation, and 95% confidence intervals  
- Summarizes validity checks (formula, space group, bond lengths, site multiplicity)  
- Supports batch retrieval for comparing multiple models efficiently  


In [3]:
import gzip
import pickle
import pandas as pd
import numpy as np
import warnings
from decifer.utility import (
    extract_numeric_property,
    get_unit_cell_volume,
    extract_volume,
)

# Suppress warnings
warnings.filterwarnings("ignore")

def calculate_statistics(df, columns, is_boolean=False):
    """
    Calculate mean, 95% confidence interval, and standard deviation for specified columns.
    """
    if is_boolean:
        df[columns] = df[columns].astype(bool)
        mean_vals = df[columns].mean() * 100
    else:
        mean_vals = df[columns].mean()

    n = len(df)
    z = 1.96  # z-score for 95% confidence
    proportions = df[columns].mean() if is_boolean else None
    ci_vals = (
        z * np.sqrt((proportions * (1 - proportions)) / n) * 100
        if is_boolean
        else None
    )
    std_vals = df[columns].std() * 100 if is_boolean else df[columns].std()

    return pd.DataFrame({
        'mean (%)' if is_boolean else 'mean': mean_vals,
        '95% CI (%)': ci_vals,
        'std (%)' if is_boolean else 'std': std_vals,
    })

def extract_validity_stats(df):
    validity_columns = [
        'formula_validity',
        'spacegroup_validity',
        'bond_length_validity',
        'site_multiplicity_validity',
        'validity',
    ]
    return calculate_statistics(df, validity_columns, is_boolean=True)

def extract_metrics_stats(df):
    metrics_columns = ['rwp', 'wd']
    return calculate_statistics(df, metrics_columns)

def process_file(file_path):
    """Process a single file and extract relevant metrics."""
    with gzip.open(file_path, 'rb') as f:
        df = pd.DataFrame(pickle.load(f))

    metrics_list = []
    for _, entry in df.iterrows():
        try:
            metrics = process_entry(entry)
            metrics_list.append(metrics)
        except Exception:
            continue

    return pd.DataFrame(metrics_list)

def process_entry(entry):
    """Process a single entry to extract metrics and validity."""
    def extract_cell_params(cif):
        a = extract_numeric_property(cif, '_cell_length_a')
        b = extract_numeric_property(cif, '_cell_length_b')
        c = extract_numeric_property(cif, '_cell_length_c')
        alpha = extract_numeric_property(cif, '_cell_angle_alpha')
        beta = extract_numeric_property(cif, '_cell_angle_beta')
        gamma = extract_numeric_property(cif, '_cell_angle_gamma')
        implied_vol = get_unit_cell_volume(a, b, c, alpha, beta, gamma)
        gen_vol = extract_volume(cif)
        return a, b, c, alpha, beta, gamma, implied_vol, gen_vol

    sample_params = extract_cell_params(entry['cif_sample'])
    gen_params = extract_cell_params(entry['cif_gen'])

    validity = {
        'formula_validity': entry['formula_validity'],
        'bond_length_validity': entry['bond_length_validity'],
        'spacegroup_validity': entry['spacegroup_validity'],
        'site_multiplicity_validity': entry['site_multiplicity_validity'],
        'validity': all(
            [
                entry['formula_validity'],
                entry['bond_length_validity'],
                entry['spacegroup_validity'],
                entry['site_multiplicity_validity'],
            ]
        ),
    }

    metrics = {
        **{f'sample_{key}': val for key, val in zip(['a', 'b', 'c', 'alpha', 'beta', 'gamma', 'implied_vol', 'gen_vol'], sample_params)},
        **{f'gen_{key}': val for key, val in zip(['a', 'b', 'c', 'alpha', 'beta', 'gamma', 'implied_vol', 'gen_vol'], gen_params)},
        **validity,
        'rwp': entry['rwp'],
        'wd': entry['wd'],
    }

    return metrics

def process_and_print_stats(file_paths):
    for path in file_paths:
        print(f"Processing: {path}")
        df = process_file(path)
        print(extract_validity_stats(df))
        print(extract_metrics_stats(df))
        print()

# File paths
file_paths = [
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer-U_(None).pkl.gz",
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer-U_(Comp).pkl.gz",
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer-U_(CompSG).pkl.gz",
    
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(None_N-0p00_B-0p05).pkl.gz",
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(Comp_N-0p00_B-0p05).pkl.gz",
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(CompSG_N-0p00_B-0p05).pkl.gz",

    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(Comp_N-0p05_B-0p05).pkl.gz",
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(Comp_N-0p00_B-0p10).pkl.gz",
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(Comp_N-0p05_B-0p10).pkl.gz",

    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(Comp_N-0p10_B-0p05).pkl.gz",
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(Comp_N-0p00_B-0p20).pkl.gz",
    # "../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_(Comp_N-0p10_B-0p20).pkl.gz",

    #"../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_chili_(Comp_N-0p00_B-0p05).pkl.gz",
    #"../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_chili_(Comp_N-0p05_B-0p10).pkl.gz",
    #"../experiments/model__conditioned_mlp_augmentation__context_3076__robust/comparison_files_fullXRD__robust/deCIFer_chili_(Comp_N-0p10_B-0p20).pkl.gz",
]

# Process and print stats
process_and_print_stats(file_paths)
